# Variants

In [1]:
from typing import List
import vcf
import pandas as pd
import re
import os

class VariantsReader:
    
    def __init__(self):
        pass

    def read_vcf(self, file: str) -> pd.DataFrame:
        reader = vcf.Reader(open(file, 'r'))
        df = pd.DataFrame([vars(r) for r in reader])
        out = df.merge(pd.DataFrame(df.INFO.tolist()),
                       left_index=True, right_index=True)
        out = out[['CHROM', 'POS', 'REF', 'ALT', 'DP', 'QUAL', 'RO', 'AO', 'INFO']]
        out['TYPE'] = out['INFO'].map(lambda x: x['TYPE'][0])
        out = out.drop('INFO', axis='columns')
        out['ALT'] = out['ALT'].map(lambda x: str(x[0]))
        out['REF'] = out['REF'].map(lambda x: str(x[0]))
        out['AO'] = out['AO'].map(lambda x: x[0])
        cols = out.columns.tolist()
        out['FILE'] = os.path.basename(file)
        out = out.reindex(columns=['FILE'] + cols)
        return out

    def read_vcfs(self, files: List[str]) -> pd.DataFrame:
        frames = [self.read_vcf(f) for f in files]
        return pd.concat(frames)

directory = 'data/snps-vcf'
files = [os.path.join(directory, f) for f in os.listdir('data/snps-vcf')]
files = [f for f in files if f.endswith('.vcf')]

vr = VariantsReader()
df = vr.read_vcfs(files)
df

,FILE,CHROM,POS,REF,ALT,DP,QUAL,RO,AO,TYPE
0,2014C-3857.filt.vcf,JASV01000001.1,16854,T,C,38,1175.580,0,38,snp
1,2014C-3857.filt.vcf,JASV01000001.1,16871,C,C,29,832.403,0,28,del
2,2014C-3857.filt.vcf,JASV01000001.1,16897,G,C,25,735.848,0,25,snp
3,2014C-3857.filt.vcf,JASV01000001.1,17327,T,TA,11,258.368,0,11,ins
4,2014C-3857.filt.vcf,JASV01000001.1,17335,G,G,13,292.065,0,13,del
...,...,...,...,...,...,...,...,...,...,...
125,2014C-3600.filt.vcf,JASV01000007.1,767386,A,G,87,2684.540,0,87,snp
126,2014C-3600.filt.vcf,JASV01000007.1,890046,T,C,96,3115.580,0,96,snp
127,2014C-3600.filt.vcf,JASV01000007.1,958303,T,G,93,3115.200,0,93,snp
128,2014C-3600.filt.vcf,JASV01000007.1,963356,A,G,23,692.768,0,23,snp


# DB model

In [2]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

Base = declarative_base()
Base

from sqlalchemy import Column, Integer, String, Sequence, BigInteger, ForeignKey

class VariationAllele(Base):
    __tablename__ = 'variation_allele'
    id = Column(Integer, primary_key=True)
    contig_id = Column(Integer, ForeignKey('reference_contig.id'))
    position = Column(BigInteger)
    ref = Column(String(255))
    alt = Column(String(255))
    var_type = Column(String(255))
    contig = relationship('ReferenceContig', back_populates='variants')
    
    def to_spdi(self):
        return f'{self.contig.contig_name}:{self.position}:{self.ref}:{self.alt}'
    
    def __repr__(self):
        return (f'<VariationAllele(id={self.id}, contig={self.contig}'
                f', position={self.position}, ref={self.ref}, alt={self.alt}), var_type={self.var_type}>')
    
    
class Reference(Base):
    __tablename__ = 'reference'
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    length = Column(Integer)
    contigs = relationship('ReferenceContig')
    
    def __repr__(self):
        return f'<Reference(id={self.id}, name={self.name}, length={self.length})>'
    
    
class ReferenceContig(Base):
    __tablename__ = 'reference_contig'
    id = Column(Integer, primary_key=True)
    ref_id = Column(Integer, ForeignKey('reference.id'))
    contig_name = Column(String(255))
    variants = relationship('VariationAllele', back_populates='contig')
    
    def __repr__(self):
        return f'<ReferenceContig(id={self.id}, contig_name={self.contig_name}, ref_id={self.ref_id})>'

# Create some data

In [3]:
from Bio import SeqIO

ref_name = '2011C-3609.fasta'
ref_length = 0
ref_contigs = {}
for record in SeqIO.parse(f"reference/{ref_name}", "fasta"):
    ref_contigs[record.id] = ReferenceContig(contig_name=record.id)
    ref_length += len(record.seq)

reference = Reference(name = ref_name, length = ref_length, contigs=list(ref_contigs.values()))
ref_contigs

{'JASV01000001.1': <ReferenceContig(id=None, contig_name=JASV01000001.1, ref_id=None)>,
 'JASV01000002.1': <ReferenceContig(id=None, contig_name=JASV01000002.1, ref_id=None)>,
 'JASV01000003.1': <ReferenceContig(id=None, contig_name=JASV01000003.1, ref_id=None)>,
 'JASV01000004.1': <ReferenceContig(id=None, contig_name=JASV01000004.1, ref_id=None)>,
 'JASV01000005.1': <ReferenceContig(id=None, contig_name=JASV01000005.1, ref_id=None)>,
 'JASV01000006.1': <ReferenceContig(id=None, contig_name=JASV01000006.1, ref_id=None)>,
 'JASV01000007.1': <ReferenceContig(id=None, contig_name=JASV01000007.1, ref_id=None)>}

# Insert into database

In [4]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///:memory:', echo=True)

Session = sessionmaker(bind=engine)
session = Session()
session

Base.metadata.create_all(engine)

2020-12-20 23:45:23,644 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-20 23:45:23,645 INFO sqlalchemy.engine.base.Engine ()
2020-12-20 23:45:23,648 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-20 23:45:23,649 INFO sqlalchemy.engine.base.Engine ()
2020-12-20 23:45:23,651 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("variation_allele")
2020-12-20 23:45:23,652 INFO sqlalchemy.engine.base.Engine ()
2020-12-20 23:45:23,654 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("variation_allele")
2020-12-20 23:45:23,655 INFO sqlalchemy.engine.base.Engine ()
2020-12-20 23:45:23,657 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("reference")
2020-12-20 23:45:23,658 INFO sqlalchemy.engine.base.Engine ()
2020-12-20 23:45:23,660 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("reference")
2020-12-20 23:45:23,661 INFO sqlalchemy.engine.base.Engine

In [5]:
session.add(reference)
session.commit()

2020-12-20 23:45:23,832 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-20 23:45:23,837 INFO sqlalchemy.engine.base.Engine INSERT INTO reference (name, length) VALUES (?, ?)
2020-12-20 23:45:23,838 INFO sqlalchemy.engine.base.Engine ('2011C-3609.fasta', 5412686)
2020-12-20 23:45:23,841 INFO sqlalchemy.engine.base.Engine INSERT INTO reference_contig (ref_id, contig_name) VALUES (?, ?)
2020-12-20 23:45:23,843 INFO sqlalchemy.engine.base.Engine (1, 'JASV01000001.1')
2020-12-20 23:45:23,847 INFO sqlalchemy.engine.base.Engine INSERT INTO reference_contig (ref_id, contig_name) VALUES (?, ?)
2020-12-20 23:45:23,848 INFO sqlalchemy.engine.base.Engine (1, 'JASV01000002.1')
2020-12-20 23:45:23,850 INFO sqlalchemy.engine.base.Engine INSERT INTO reference_contig (ref_id, contig_name) VALUES (?, ?)
2020-12-20 23:45:23,851 INFO sqlalchemy.engine.base.Engine (1, 'JASV01000003.1')
2020-12-20 23:45:23,855 INFO sqlalchemy.engine.base.Engine INSERT INTO reference_contig (ref_id, contig_name) 

In [6]:
ref = session.query(Reference).filter_by(id = 1).first()
ref_contigs = {c.contig_name: c for c in ref.contigs}
ref_contigs

2020-12-20 23:45:24,003 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-20 23:45:24,006 INFO sqlalchemy.engine.base.Engine SELECT reference.id AS reference_id, reference.name AS reference_name, reference.length AS reference_length 
FROM reference 
WHERE reference.id = ?
 LIMIT ? OFFSET ?
2020-12-20 23:45:24,009 INFO sqlalchemy.engine.base.Engine (1, 1, 0)
2020-12-20 23:45:24,016 INFO sqlalchemy.engine.base.Engine SELECT reference_contig.id AS reference_contig_id, reference_contig.ref_id AS reference_contig_ref_id, reference_contig.contig_name AS reference_contig_contig_name 
FROM reference_contig 
WHERE ? = reference_contig.ref_id
2020-12-20 23:45:24,018 INFO sqlalchemy.engine.base.Engine (1,)


{'JASV01000001.1': <ReferenceContig(id=1, contig_name=JASV01000001.1, ref_id=1)>,
 'JASV01000002.1': <ReferenceContig(id=2, contig_name=JASV01000002.1, ref_id=1)>,
 'JASV01000003.1': <ReferenceContig(id=3, contig_name=JASV01000003.1, ref_id=1)>,
 'JASV01000004.1': <ReferenceContig(id=4, contig_name=JASV01000004.1, ref_id=1)>,
 'JASV01000005.1': <ReferenceContig(id=5, contig_name=JASV01000005.1, ref_id=1)>,
 'JASV01000006.1': <ReferenceContig(id=6, contig_name=JASV01000006.1, ref_id=1)>,
 'JASV01000007.1': <ReferenceContig(id=7, contig_name=JASV01000007.1, ref_id=1)>}

In [7]:
df2 = df.rename({
    'CHROM': 'sequence_name',
    'POS': 'position',
    'REF': 'ref',
    'ALT': 'alt',
    'TYPE': 'var_type'
}, axis='columns').drop(['DP', 'QUAL', 'RO', 'AO'], axis='columns')

df2 = df2[df2['FILE'] == '2014C-3857.filt.vcf']
df2 = df2.drop('FILE', axis='columns')
df2['contig'] = df2['sequence_name'].apply(lambda x: ref_contigs[x])
df2 = df2.drop('sequence_name', axis='columns')
variants = df2.apply(lambda x: VariationAllele(**x), axis='columns').tolist()
variants[0]

<VariationAllele(id=None, contig=<ReferenceContig(id=1, contig_name=JASV01000001.1, ref_id=1)>, position=16854, ref=T, alt=C), var_type=snp>

In [8]:
#session.bulk_insert_mappings(VariationAllele, df2.to_dict(orient="records"))
session.bulk_save_objects(variants)
session.commit()

2020-12-20 23:45:24,716 INFO sqlalchemy.engine.base.Engine INSERT INTO variation_allele (position, ref, alt, var_type) VALUES (?, ?, ?, ?)
2020-12-20 23:45:24,723 INFO sqlalchemy.engine.base.Engine ((16854, 'T', 'C', 'snp'), (16871, 'C', 'C', 'del'), (16897, 'G', 'C', 'snp'), (17327, 'T', 'TA', 'ins'), (17335, 'G', 'G', 'del'), (17347, 'C', 'C', 'del'), (17360, 'T', 'A', 'snp'), (17468, 'T', 'C', 'snp')  ... displaying 10 of 133 total bound parameter sets ...  (975650, 'T', 'G', 'snp'), (1014501, 'A', 'G', 'snp'))
2020-12-20 23:45:24,736 INFO sqlalchemy.engine.base.Engine INSERT INTO variation_allele (contig_id, position, ref, alt, var_type) VALUES (?, ?, ?, ?, ?)
2020-12-20 23:45:24,739 INFO sqlalchemy.engine.base.Engine (1, 16854, 'T', 'C', 'snp')
2020-12-20 23:45:24,741 INFO sqlalchemy.engine.base.Engine INSERT INTO variation_allele (contig_id, position, ref, alt, var_type) VALUES (?, ?, ?, ?, ?)
2020-12-20 23:45:24,743 INFO sqlalchemy.engine.base.Engine (1, 16871, 'C', 'C', 'del')


In [9]:
v = session.query(VariationAllele).filter_by(position = 16854).all()
v

2020-12-20 23:45:25,258 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-20 23:45:25,260 INFO sqlalchemy.engine.base.Engine SELECT variation_allele.id AS variation_allele_id, variation_allele.contig_id AS variation_allele_contig_id, variation_allele.position AS variation_allele_position, variation_allele.ref AS variation_allele_ref, variation_allele.alt AS variation_allele_alt, variation_allele.var_type AS variation_allele_var_type 
FROM variation_allele 
WHERE variation_allele.position = ?
2020-12-20 23:45:25,262 INFO sqlalchemy.engine.base.Engine (16854,)
2020-12-20 23:45:25,266 INFO sqlalchemy.engine.base.Engine SELECT reference_contig.id AS reference_contig_id, reference_contig.ref_id AS reference_contig_ref_id, reference_contig.contig_name AS reference_contig_contig_name 
FROM reference_contig 
WHERE reference_contig.id = ?
2020-12-20 23:45:25,267 INFO sqlalchemy.engine.base.Engine (1,)


[<VariationAllele(id=1, contig=None, position=16854, ref=T, alt=C), var_type=snp>,
 <VariationAllele(id=134, contig=<ReferenceContig(id=1, contig_name=JASV01000001.1, ref_id=1)>, position=16854, ref=T, alt=C), var_type=snp>]

In [17]:
r = session.query(ReferenceContig).filter_by(contig_name = 'JASV01000004.1').first()
r

2020-12-20 23:46:40,015 INFO sqlalchemy.engine.base.Engine SELECT reference_contig.id AS reference_contig_id, reference_contig.ref_id AS reference_contig_ref_id, reference_contig.contig_name AS reference_contig_contig_name 
FROM reference_contig 
WHERE reference_contig.contig_name = ?
 LIMIT ? OFFSET ?
2020-12-20 23:46:40,018 INFO sqlalchemy.engine.base.Engine ('JASV01000004.1', 1, 0)


<ReferenceContig(id=4, contig_name=JASV01000004.1, ref_id=1)>

In [18]:
[v.to_spdi() for v in r.variants]

2020-12-20 23:46:42,032 INFO sqlalchemy.engine.base.Engine SELECT variation_allele.id AS variation_allele_id, variation_allele.contig_id AS variation_allele_contig_id, variation_allele.position AS variation_allele_position, variation_allele.ref AS variation_allele_ref, variation_allele.alt AS variation_allele_alt, variation_allele.var_type AS variation_allele_var_type 
FROM variation_allele 
WHERE ? = variation_allele.contig_id
2020-12-20 23:46:42,037 INFO sqlalchemy.engine.base.Engine (4,)


['JASV01000004.1:46893:A:T',
 'JASV01000004.1:266363:A:C',
 'JASV01000004.1:302849:A:G',
 'JASV01000004.1:311149:G:A',
 'JASV01000004.1:316700:G:A',
 'JASV01000004.1:427335:A:G',
 'JASV01000004.1:448776:C:A',
 'JASV01000004.1:464957:C:T',
 'JASV01000004.1:560978:A:G',
 'JASV01000004.1:619931:A:G',
 'JASV01000004.1:630889:A:G',
 'JASV01000004.1:667921:A:G',
 'JASV01000004.1:668427:G:A',
 'JASV01000004.1:669669:G:A',
 'JASV01000004.1:698623:G:T',
 'JASV01000004.1:738037:G:A',
 'JASV01000004.1:742706:T:C',
 'JASV01000004.1:771493:C:T',
 'JASV01000004.1:795516:G:A',
 'JASV01000004.1:835628:A:G',
 'JASV01000004.1:963212:G:A',
 'JASV01000004.1:972449:T:C',
 'JASV01000004.1:1058559:G:A',
 'JASV01000004.1:1072253:G:A',
 'JASV01000004.1:1152591:A:G',
 'JASV01000004.1:1196181:T:C',
 'JASV01000004.1:1250576:G:A',
 'JASV01000004.1:1297534:T:G',
 'JASV01000004.1:1369978:A:AT',
 'JASV01000004.1:1384749:A:C',
 'JASV01000004.1:1454246:G:A',
 'JASV01000004.1:1468175:G:A',
 'JASV01000004.1:1468354:C:T',